<a href="https://colab.research.google.com/github/therishabhmittal-05/NLP/blob/main/GenerateNamesRnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import unicodedata
import string
import glob
from io import open
import os

In [14]:
all_chars = string.ascii_letters + " ,.;'-"
n_letters = len(all_chars) + 1

In [15]:
def findFiles(path): return glob.glob(path)

In [16]:
def unicode_ascii(s):
  return "".join(
      c for c in unicodedata.normalize("NFD", s)
      if unicodedata.category(c) != 'Mn'
      and c in all_chars
  )

In [17]:
def readFiles(filename):
  with open (filename, encoding='utf-8') as file:
    return [unicode_ascii(line.strip()) for line in file]

In [19]:
category_dict = {}
all_categories= []

for filename in findFiles('/content/drive/MyDrive/Colab Notebooks/data/data/names/*.txt'):
  category = os.path.splitext(os.path.basename(filename))[0]
  all_categories.append(category)
  lines = readFiles(filename)
  category_dict[category] = lines

n_cat = len(all_categories)

In [25]:
print(len(category_dict["Russian"]))

9408


In [28]:
class GenRNN(nn.Module):
  def __init__(self, ip_size, op_size, h_size):
    super(GenRNN, self).__init__()
    self.h_size = h_size

    self.i2h = nn.Linear(n_cat + ip_size + h_size, h_size)
    self.i2o = nn.Linear(n_cat + ip_size + h_size, op_size)
    self.o2o = nn.Linear(h_size+op_size, op_size)
    self.dropout = nn.Dropout(0.1)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, category, input, hidden):
    ip_combined = torch.cat((category, input, hidden),1)
    hidden = self.i2h(ip_combined)
    output = self.i2o(ip_combined)

    op_combined = torch.cat((hidden, output),1)
    output = self.o20(op_combined)
    output = self.dropout(output)
    output = self.softmax(output)
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, self.h_size)